<a href="https://colab.research.google.com/github/akaraci/LargeDatasetCovid19/blob/main/Multi_Class_Pre_Trained_and_Modified_Pre_Trained_Models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autotime
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax
#from sklearn.metrics import multilabel_confusion_matrix
from sklearn import metrics
from google.colab import drive
drive.mount('/content/drive')
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.callbacks import EarlyStopping
np.random.seed(1000)

from keras.applications.vgg19 import VGG19,preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50

from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import  ImageDataGenerator, img_to_array, load_img
from glob import glob

In [ ]:
#------------Modified and unmodified Pre-trained Architectures's (VGG16, VGG19, InceptionV3 and Resnet50) Multi Class Code ---------------- 
from sklearn.preprocessing import label_binarize
from itertools import cycle

X=pickle.load(open("drive/My Drive/LargeDataSetBen/X_databen_three_az.pickle", "rb"))
y=pickle.load(open("drive/My Drive/LargeDataSetBen/Y_databen_three_az.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0  
y = to_categorical(y, num_classes = 3)

#Unmodified Pre-trained Architectures
def buildmodel_pretrained_architecture():
  base_model=InceptionV3(include_top=False, weights="imagenet", input_shape=(224,224,3))
  #base_model=ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3))
  #base_model=VGG19(include_top=False,weights='imagenet', input_shape=(224,224,3))
  #base_model=VGG16(include_top=False,weights='imagenet', input_shape=(224,224,3))
  base_model.trainable=True
  model=Sequential()
  model.add(base_model)
  model.add(GlobalAveragePooling2D())  #for InceptionV3 and Resnet50
  """model.add(Flatten())   #for VGG16 and VGG19
  model.add(Dense(4096))
  model.add(Dense(4096))"""
  numberOfClass=3
  model.add(Dense(numberOfClass,activation='softmax'))
  return model

#Modified Pre-trained Architectures
def buildmodel_modified_pretrained_architecture():
  #base_model=InceptionV3(include_top=False, weights="imagenet", input_shape=(224,224,3))
  #base_model=ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3))
  #base_model=VGG19(include_top=False,weights='imagenet', input_shape=(224,224,3))
  base_model=VGG16(include_top=False,weights='imagenet', input_shape=(224,224,3))
  base_model.trainable=True
  model=Sequential()
  model.add(base_model)
  model.add(Flatten())
  model.add(Dense(64))
  model.add(Dense(128))
  model.add(Dense(256))
  numberOfClass=3
  model.add(Dense(numberOfClass,activation='softmax'))
  return model

k=1
if (k==0):opt =tf.optimizers.Adam(lr=0.001)
if (k==1):opt=tf.optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=None, decay=0.0)
if (k==2):opt = tf.optimizers.SGD(lr=0.01, clipnorm=1.)
if (k==3):opt =tf.optimizers.RMSprop(lr=0.001, rho=0.8, epsilon=None, decay=0.0)
if (k==4):opt=tf.optimizers.Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
if (k==5):opt=tf.optimizers.Nadam(lr=0.02, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

kacfold=5
from sklearn.model_selection import KFold
kf = KFold(n_splits=kacfold, shuffle=False)
sumaccuracy=0
sumrecall=0
sumprecision=0
sumf1=0
owerlapconf_mat=0
history=[]
acc_val=[]
result=[]
plt.figure(figsize=(10, 10))
fold=1
#opt=keras.optimizers.Adam(lr=0.001)
for train_index, test_index in kf.split(X):
    
    x_train = X[train_index]
    y_train = y[train_index]
    x_test = X[test_index]
    y_test = y[test_index]
    model=buildmodel_modified_pretrained_architecture()
    #model=buildmodel_pretrained_architecture()
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    history = model.fit(x_train, y_train, batch_size=32, epochs=50, verbose=0) 
    np.save("drive/My Drive/LargeDataSetBen/modelsmodify/my_history_deneme"+str(fold)+".npy",history.history)
    prediction = model.predict(x_test)
    Y_pred_classes = np.argmax(prediction,axis = 1) 
    y_test_classes = np.argmax(y_test,axis = 1)
    result=metrics.classification_report(y_test_classes, Y_pred_classes, digits=4,output_dict=True)
    result2=metrics.classification_report(y_test_classes, Y_pred_classes, digits=4,output_dict=False)
    sumaccuracy=sumaccuracy+result['accuracy']
    sumrecall=sumrecall+result['weighted avg']['recall']
    sumprecision=sumprecision+result['weighted avg']['precision']
    sumf1=sumf1+result['weighted avg']['f1-score']
    conf_mat =confusion_matrix(y_test_classes, Y_pred_classes, labels=[0, 1, 2])
    owerlapconf_mat=owerlapconf_mat+conf_mat
    print('\n',conf_mat)
    print(result2)
    fold=fold+1
meanaccuracy=sumaccuracy/kacfold
print("Mean Accuracy=",meanaccuracy)
print("Mean weighted avg Recall=",sumrecall/kacfold)
print("Mean weighted avg Precision=",sumprecision/kacfold)
print("Mean weighted avg F1-Score=",sumf1/kacfold)
print('\nOwerlap Confusion Matrix:\n',owerlapconf_mat)
total1=sum(sum(owerlapconf_mat))
owrecallnormal=owerlapconf_mat[0,0]/(owerlapconf_mat[0,0]+owerlapconf_mat[0,1]+owerlapconf_mat[0,2])
owrecallcovid=owerlapconf_mat[1,1]/(owerlapconf_mat[1,1]+owerlapconf_mat[1,0]+owerlapconf_mat[1,2])
owrecallpneumonia=owerlapconf_mat[2,2]/(owerlapconf_mat[2,2]+owerlapconf_mat[2,0]+owerlapconf_mat[2,1])

owprecisionnormal=owerlapconf_mat[0,0]/(owerlapconf_mat[0,0]+owerlapconf_mat[1,0]+owerlapconf_mat[2,0])
owprecisioncovid=owerlapconf_mat[1,1]/(owerlapconf_mat[1,1]+owerlapconf_mat[0,1]+owerlapconf_mat[2,1])
owprecisionpneumonia=owerlapconf_mat[2,2]/(owerlapconf_mat[2,2]+owerlapconf_mat[1,2]+owerlapconf_mat[0,2])

owf1normal=2*(owrecallnormal*owprecisionnormal)/(owrecallnormal+owprecisionnormal)
owf1covid=2*(owrecallcovid*owprecisioncovid)/(owrecallcovid+owprecisioncovid)
owf1pneumonia=2*(owrecallpneumonia*owprecisionpneumonia)/(owrecallpneumonia+owprecisionpneumonia)

print("Owerlap Recall Normal=",owrecallnormal)
print("Owerlap Recall Covid=",owrecallcovid)
print("Owerlap Recall Pneumonia=",owrecallpneumonia)

print("Owerlap Precision Normal=",owprecisionnormal)
print("Owerlap Precision Covid=",owprecisioncovid)
print("Owerlap Precision Pneumonia=",owprecisionpneumonia)


print("Owerlap  F1-Score Normal=",owf1normal)
print("Owerlap  F1-Score Covid=",owf1covid)
print("Owerlap  F1-Score Pneumonia=",owf1pneumonia)


In [ ]:
#Plot Accuracy and Loss Curve
#%load_ext autotime
import json,codecs
import pandas as pd
import numpy as np

n=np.load("drive/My Drive/LargeDataSetBen/modelsmodify/my_historyk2.npy",allow_pickle='TRUE').item()
plt.plot(n['loss'],label='train loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend()
plt.show()
plt.plot(n['accuracy'],label='train acc')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend()
plt.show()

In [ ]:
pip install scikit-plot

In [ ]:
!pip install ipython-autotime